****ARIMA forecasts for electricity prices dataset of queensland for the year 2018.

In [ ]:
#import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import utils
import os
from datetime import datetime, timedelta

We will write two other scripts, one for parameter tuning and other for fetching data from AEMO website so that we dont have to deal with too many csv files, in this script we will use statistics and theory to determine the order of ARIMA model.

In [ ]:
#dataset
prices = pd.read_csv('C:\\Users\\dell i\\Desktop\\Academics\\MTech Project\\price forecasting\\data\\prices_dataset.csv')
prices['SETTLEMENTDATE'] = pd.to_datetime(prices['SETTLEMENTDATE'])
prices = prices.set_index(['SETTLEMENTDATE'])

In [ ]:
prices.describe()

In [ ]:
prices.head(10)

We will try seasonal differencing of data and see the rolling statistics and the distribution of data.


In [ ]:
#seasonal differencing by 24 hours, 48 observations and plotting it
start_date = datetime(2018,10,5,0,30,0)
period = 10  # observing prices for a period of 10 days
end_date = start_date + timedelta(days = period)
diff_price_series = prices.diff(periods = 48)
diff_price_series.dropna()[start_date:end_date].plot(figsize = (22,10))
plt.legend()
plt.show()


In [ ]:
#Plotting the rolling statistics of the differenced series, 
utils.plot_rolling_stats(diff_price_series.dropna()[start_date:end_date])

In [ ]:
#ACF and PACF plots of differenced searies
utils.plot_acf_pacf(diff_price_series,n_lags = 120)

Lets make an SARIMA model as follows and restrict our parameter grid for grid search within this range only
p- number of auto regressivc terms, 8 as the PACF lags are not significant
d - non seasonal differences, 0 as no non seasonal difference is there
q - number of moving average terms, 8 as the ACF lags are not significant after that
P - number of seasonal auto regressive terms, 1 from the PACF plot(after 48th lag, there are not significant lags)
D- seasonal differences, 1
Q - number of seasonal moving average terms, 1 from the ACF plot
S- 48
therefore
(p,d,q) = (8,0,8) or (8,1,8) 
(P,D,Q,S) = (1,1,1,48)


In [ ]:
#fitting the model
model=SARIMAX(prices, order=(8, 0, 8), seasonal_order=(1, 1,1, 48)).fit(disp=-1)